### Importing Libraries

In [25]:
import asyncio,yaml,json
from typing import TYPE_CHECKING, Annotated
from utils import *
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
import logging,re
from semantic_kernel.functions import KernelFunctionFromPrompt
from openai import AzureOpenAI
from utils import *
from Plugins.searchplugin import SearchPlugin
from Plugins.emailplugin import EmailPlugin
# logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


# Search Agent

### Initialize kernel

In [30]:

##initializing kernel
search_agent_kernel = Kernel()

## adding model as service to kernel
sk_client = AzureChatCompletion(async_client=get_openai_client(), service_id="4o-agent", deployment_name="gpt-4o")
search_agent_kernel.add_service(service=sk_client)


**Adding plugins**

SearchAgent requires 2 plugins

1. Search Plugin: to search stock related news
2. Time Plugin: to calculate date and time

In [31]:
from Plugins.searchplugin import SearchPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin


search_plugin = SearchPlugin()
time_plugin = TimePlugin()

## adding plugins to kernel
search_agent_kernel.add_plugin(plugin=search_plugin,plugin_name="SearchPlugin")
search_agent_kernel.add_plugin(plugin=time_plugin,plugin_name="TimePlugin")

KernelPlugin(name='TimePlugin', description=None, functions={'date': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='date', plugin_name='TimePlugin', description='Get the current date.', parameters=[], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x11d8fdda0>, streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x11d8fde10>, method=<bound method TimePlugin.date of TimePlugin()>, stream_method=None), 'date_matching_last_day_name': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='date_matching_last_day_name', plugin_name='TimePlugin', description='Get the date of the last day matc

**Lets print all plugins**

In [32]:
for plugin_name, plugin in search_agent_kernel.plugins.items():
    for function_name, function in plugin.functions.items():
        print(f"Plugin: {plugin_name}, Function: {function_name}")

Plugin: SearchPlugin, Function: SearchCode
Plugin: SearchPlugin, Function: SearchNews
Plugin: TimePlugin, Function: date
Plugin: TimePlugin, Function: date_matching_last_day_name
Plugin: TimePlugin, Function: day
Plugin: TimePlugin, Function: dayOfWeek
Plugin: TimePlugin, Function: days_ago
Plugin: TimePlugin, Function: hour
Plugin: TimePlugin, Function: hourNumber
Plugin: TimePlugin, Function: iso_date
Plugin: TimePlugin, Function: minute
Plugin: TimePlugin, Function: month
Plugin: TimePlugin, Function: month_number
Plugin: TimePlugin, Function: now
Plugin: TimePlugin, Function: second
Plugin: TimePlugin, Function: time
Plugin: TimePlugin, Function: timeZoneName
Plugin: TimePlugin, Function: timeZoneOffset
Plugin: TimePlugin, Function: today
Plugin: TimePlugin, Function: utcNow
Plugin: TimePlugin, Function: year


**Get Execution settings**

In [33]:
## Lets get execution settings for agent
## Add execution settings of the kernel 
search_agent_execution_settings = search_agent_kernel.get_prompt_execution_settings_from_service_id(service_id="4o-agent")
# Configure the function choice behavior to auto invoke kernel functions
search_agent_execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
search_agent_execution_settings.parallel_tool_calls = False

In [34]:


# define agent
AGENT_NAME = "SearchAgent"
AGENT_INSTRUCTIONS = """

Role:
You are a financial department agent specializing in stock price analysis and related tasks.

Plugins:

SearchPlugin:

SearchCode: Retrieves a company code based on a name of the company. This code is used as input for SearchNews.
SearchNews: Fetches financial news using a company code and date range (from_date, to_date).

TimePlugin: Manages time-related operations, including retrieving the current date.

Instructions:

Time Management:

Use TimePlugin for the current date in YYYY-MM-DD format.
Last Week: Set from_date to Monday and to_date to Friday of the previous week.
Last Month: Set from_date to the 1st and to_date to the last day of the previous month.
No Time Frame Mentioned: Use today’s date as from_date.


Company code: 
Search Code will returns a company code if the user mentions a company in the query.
This company code gets passed to SearchNews


Accuracy:
Never assume or fabricate information.
Always include the time frame in the final output.
"""



#Create the agent

search_agent = ChatCompletionAgent(
    service_id="4o-agent",
    kernel=search_agent_kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings=search_agent_execution_settings))

##"excluded_plugins": ["PluginName"]}

## assigning a chat history object
chat_history = ChatHistory()

## function to chat
async def chat(user_message,agent):
    chat_history.add_user_message(user_message)
    async for response in agent.invoke(history=chat_history,verbose=True):
            print(f"{response.content}")
    chat_history.add_assistant_message(str(response))

In [35]:
search_agent.kernel.plugins

{'SearchPlugin': KernelPlugin(name='SearchPlugin', description=None, functions={'SearchCode': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='SearchCode', plugin_name='SearchPlugin', description='Extrats company name from user query and returns corresponding code', parameters=[KernelParameterMetadata(name='company_name', description='name of the company', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'name of the company'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='code of company in stockmarket', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'code of company in stockmarket'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyH

In [36]:
user_query = "what happend to google stocks last week"


response = await chat(user_query,agent=search_agent)

DEBUG:semantic_kernel.agents.chat_completion.chat_completion_agent:[ChatCompletionAgent] Invoking AzureChatCompletion.
DEBUG:msal.authority:Initializing with Entra authority: https://login.microsoftonline.com/0ab4cbbf-4bc7-4826-b52c-a14fed5286b9
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://login.microsoftonline.com/0ab4cbbf-4bc7-4826-b52c-a14fed5286b9/v2.0/.well-known/openid-configuration'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.13.0 (macOS-10.16-x86_64-i386-64bit-Mach-O)'
No body was attached to the request
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): login.microsoftonline.com:443


DEBUG:urllib3.connectionpool:https://login.microsoftonline.com:443 "GET /0ab4cbbf-4bc7-4826-b52c-a14fed5286b9/v2.0/.well-known/openid-configuration HTTP/11" 200 1753
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Cache-Control': 'max-age=86400, private'
    'Content-Type': 'application/json; charset=utf-8'
    'Strict-Transport-Security': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Access-Control-Allow-Origin': 'REDACTED'
    'Access-Control-Allow-Methods': 'REDACTED'
    'P3P': 'REDACTED'
    'x-ms-request-id': '5dbe9503-6a9c-46be-a682-c36c9ed83800'
    'x-ms-ests-server': 'REDACTED'
    'x-ms-srs': 'REDACTED'
    'Content-Security-Policy-Report-Only': 'REDACTED'
    'X-XSS-Protection': 'REDACTED'
    'Set-Cookie': 'REDACTED'
    'Date': 'Sat, 22 Feb 2025 18:38:04 GMT'
    'Content-Length': '1753'
DEBUG:msal.authority:openid_config("https://login.microsoftonline.com/0ab4cbbf-4bc7-4826-b52c-a14fed5286b9/v2.0/.well-known/op

During the last week (from February 17, 2025, to February 21, 2025), several factors influenced Google's stock (Alphabet Inc.).

1. **Investor Caution**: There was a cautious sentiment towards Alphabet's stock due to rising capital expenditures and market competition. The focus on AI and cloud service expansion necessitates significant investment, impacting short-term profitability.

2. **AI Ambitions**: Alphabet's ambitious plans in the AI sector come with substantial financial implications. The substantial capital needed for AI technology advancements raises concerns about the return on investment and market adoption.

3. **Market Position**: Traditional giants like Alphabet are facing challenges in maintaining their market positions amidst increased competition and regulatory scrutiny, influencing strategic decisions in AI and cloud services.

4. **Financial Strategies**: Elevated capital expenditures related to AI and cloud infrastructure are under investor scrutiny. Balancing inno

In [37]:
user_query = "how is facebook performing?"


response = await chat(user_query,agent=search_agent)

DEBUG:semantic_kernel.agents.chat_completion.chat_completion_agent:[ChatCompletionAgent] Invoking AzureChatCompletion.
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/deployments/gpt-4o/chat/completions', 'headers': {'Authorization': '<redacted>'}, 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': '\n\nRole:\nYou are a financial department agent specializing in stock price analysis and related tasks.\n\nPlugins:\n\nSearchPlugin:\n\nSearchCode: Retrieves a company code based on a name of the company. This code is used as input for SearchNews.\nSearchNews: Fetches financial news using a company code and date range (from_date, to_date).\n\nTimePlugin: Manages time-related operations, including retrieving the current date.\n\nInstructions:\n\nTime Management:\n\nUse TimePlugin for the current date in YYYY-MM-DD format.\nLast Week: Set from_date to Monday and to_date to Friday of the previous week.\nLast Month: Set from_date to the 1st and to_dat

As of today, February 22, 2025, here are the key developments impacting Facebook (Meta):

1. **Partnership with Healthcare**: Meta has entered into a partnership with a major healthcare provider aimed at developing innovative health solutions using AI and data analytics. This move exemplifies Meta's efforts to leverage its technology in diverse sectors and expand its influence beyond social media.

2. **Regulatory Scrutiny**: The company is currently facing increased regulatory scrutiny over its data privacy practices. Meta is actively working to address these regulatory concerns, which could influence investor sentiment and impact its market performance.

These recent activities suggest an ongoing strategic diversification while dealing with regulatory challenges.


# Email Agent



### Initialize kernel



In [38]:

##initializing kernel
email_agent_kernel = Kernel()

## adding model as service to kernel
sk_client = AzureChatCompletion(async_client=get_openai_client(), service_id="4o-agent", deployment_name="gpt-4o")
email_agent_kernel.add_service(service=sk_client)


In [39]:
from Plugins.emailplugin import EmailPlugin

email_plugin = EmailPlugin()
email_agent_kernel.add_plugin(plugin=email_plugin,plugin_name="EmailPlugin")

KernelPlugin(name='EmailPlugin', description=None, functions={'EmailComposer': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='EmailComposer', plugin_name='EmailPlugin', description='Generates subject and body of email based on content', parameters=[KernelParameterMetadata(name='content', description='content based on which email is generated', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'content based on which email is generated'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='subject and body of email', default_value=None, type_='dict[dict]', is_required=True, type_object=<class 'dict'>, schema_data={'type': 'object', 'description': 'subject and body of email'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrumen

In [41]:
## Lets get execution settings for agent
## Add execution settings of the kernel 
email_agent_execution_settings = email_agent_kernel.get_prompt_execution_settings_from_service_id(service_id="4o-agent")
# Configure the function choice behavior to auto invoke kernel functions
email_agent_execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
## there is filter parameter we can use if needed eg:filters={"included_plugins": ["EmailPlugin"]}
email_agent_execution_settings.parallel_tool_calls = False

In [43]:
AGENT_NAME= "EmailWriter"

AGENT_INSTRUCTIONS = """ 


Act as a financial department agent specializing in writing and sending emails. Use the following plugins to handle all email-related tasks:  

- **EmailComposer:** Composes emails based on provided content.  
- **EmailValidator:** Validates the recipient's email address.  
- **EmailSender:** Sends the composed email after validation.  

**Instructions:**  
1. Prompt the user for the recipient's email if not provided.  
2. Always validate the email using EmailValidator before sending.  
3. If validation fails, ask the user to correct the email address before proceeding.
4. You must try to fetch recipient name from email address provided.
5. send email only if user wants to send it.Otherwise ask for confirmation 

"""

# Create the agent

email_agent = ChatCompletionAgent(
    service_id="4o-agent",
    kernel=email_agent_kernel,
    description="Select me when you want to send email",
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings=email_agent_execution_settings))



In [45]:
news = """
In January 2025, several significant developments influenced the stock market, particularly concerning Alphabet (Google's parent company):

1. **Alphabet's 'Challenging' 2025 Outlook**: Alphabet's CEO highlighted that 2025 will be a pivotal yet challenging year, focusing on AI advancements and infrastructure investments.

2. **Google's Quantum Computing Milestone**: Alphabet announced a major breakthrough in quantum computing, which could transform future technologies and boost investor confidence.

3. **Google Stock Rises on AI Growth**: Google’s stock rose by 8% early in 2025, primarily driven by new AI product launches and strong cost management.

4. **Alphabet's Cloud Expansion Continues**: Analysts noted the continued expansion of Alphabet's cloud services as a significant contributor to its robust financial performance.

5. **Analyst Downgrades Alphabet Amid Antitrust**: JMP Securities downgraded Alphabet due to potential impacts of ongoing antitrust proceedings that could affect search distribution and revenue.

Overall, there was a mix of optimism due to technological advancements and concerns over potential antitrust implications shaping the market sentiment for Alphabet in January 2025.
"""


user_query = f"Hey am Anna.Please compose an email based mentioning about following stock related news: {news}"


response = await chat(user_query,agent=email_agent)


DEBUG:semantic_kernel.agents.chat_completion.chat_completion_agent:[ChatCompletionAgent] Invoking AzureChatCompletion.
DEBUG:msal.authority:Initializing with Entra authority: https://login.microsoftonline.com/0ab4cbbf-4bc7-4826-b52c-a14fed5286b9
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://login.microsoftonline.com/0ab4cbbf-4bc7-4826-b52c-a14fed5286b9/v2.0/.well-known/openid-configuration'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.13.0 (macOS-10.16-x86_64-i386-64bit-Mach-O)'
No body was attached to the request
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): login.microsoftonline.com:443


DEBUG:urllib3.connectionpool:https://login.microsoftonline.com:443 "GET /0ab4cbbf-4bc7-4826-b52c-a14fed5286b9/v2.0/.well-known/openid-configuration HTTP/11" 200 1753
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Cache-Control': 'max-age=86400, private'
    'Content-Type': 'application/json; charset=utf-8'
    'Strict-Transport-Security': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Access-Control-Allow-Origin': 'REDACTED'
    'Access-Control-Allow-Methods': 'REDACTED'
    'P3P': 'REDACTED'
    'x-ms-request-id': 'b307ec3e-0c05-46e9-889e-18efd19b2f00'
    'x-ms-ests-server': 'REDACTED'
    'x-ms-srs': 'REDACTED'
    'Content-Security-Policy-Report-Only': 'REDACTED'
    'X-XSS-Protection': 'REDACTED'
    'Set-Cookie': 'REDACTED'
    'Date': 'Sat, 22 Feb 2025 18:40:06 GMT'
    'Content-Length': '1753'
DEBUG:msal.authority:openid_config("https://login.microsoftonline.com/0ab4cbbf-4bc7-4826-b52c-a14fed5286b9/v2.0/.well-known/op

Could you please provide the recipient's email address?


In [47]:
user_query = f"ok got it. his address is aronjacob@gmail.com"


response = await chat(user_query,agent=email_agent)
print(response)

DEBUG:semantic_kernel.agents.chat_completion.chat_completion_agent:[ChatCompletionAgent] Invoking AzureChatCompletion.
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/deployments/gpt-4o/chat/completions', 'headers': {'Authorization': '<redacted>'}, 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': " \n\n\nAct as a financial department agent specializing in writing and sending emails. Use the following plugins to handle all email-related tasks:  \n\n- **EmailComposer:** Composes emails based on provided content.  \n- **EmailValidator:** Validates the recipient's email address.  \n- **EmailSender:** Sends the composed email after validation.  \n\n**Instructions:**  \n1. Prompt the user for the recipient's email if not provided.  \n2. Always validate the email using EmailValidator before sending.  \n3. If validation fails, ask the user to correct the email address before proceeding.\n4. You must try to fetch recipient name from email address pr

The email has been composed with the following details:

- **Subject**: Navigating Alphabet's Transformative Journey in 2025
- **Body**:
  ```
  Hi Aron Jacob,

  I hope this email finds you well! I wanted to share some interesting updates regarding Alphabet's (Google's parent company) journey in the stock market as we step into 2025. It's an exciting time with a few twists and turns worth noting.

  Alphabet's CEO has been quite open about the challenges and opportunities this year presents, with a strong focus on AI advancements and infrastructure investments. This focus seems to be paying off, particularly with a groundbreaking achievement in quantum computing that promises to reshape future technologies and could potentially strengthen investor confidence.

  Furthermore, Alphabet's cloud services continue to show impressive growth, contributing positively to its overall financial health. This, combined with new AI product launches and prudent cost management, has helped Google's s

## MultiAgent

In [1]:
from semantic_kernel.agents import AgentGroupChat
